# Query signals generation test report

This Jupyter Notebook captures the unit test results of the detection rules documents emitter.
Here you can learn what kind of queries the emitter handles and the documents it generates.

Curious about the inner workings? Read [here](signals_generation.md). Need help in using a Jupyter Notebook?
Read [here](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html#structure-of-a-notebook-document).

## Table of contents
 1. [Preliminaries](#Preliminaries)
 1. [Simple queries](#Simple-queries)
 1. [Sequence queries](#Sequence-queries)
 1. [Error conditions](#Error-conditions)
 1. [Any oddities?](#Any-oddities?)

## Preliminaries

This is an auxiliary cell, it prepares the environment for all the subsequent cells. It's also
a simple example of emitter API usage.

In [ ]:
import os; os.chdir('../..')  # use the repo's root as base for local modules import
import eql
from detection_rules.events_emitter import emitter

def emit(query):
    with eql.parser.elasticsearch_syntax:
        try:
            return emitter.emit_docs(emitter.emit(eql.parse_query(query)))
        except Exception as e:
            print(e)

## How to read the test results

If you opened this as freshly generated, the output cells content comes from the unit tests run and
you can read it as a plain test report. Such content is generated in a controlled environment and is
meant not to change between unit tests runs.
The notebook itself does not run in such controlled environment therefore executing these cells, even
if unmodified, will likely lead to different results each time.

On the other hand, you can experiment and modify the queries in the input cells, check the results
and, why not?, report any interesting finding. You can also add and remove cells at will.

## Simple queries

What follow are all queries that may trigger a signal just with a single _minimal matching document_,
therefore at most one document is generated for each execution.

You will notice that some queries actually generate multiple documents, this happens when
the query is disjunctive (e.g. contains an _or_ operator). In these cases each of the generated
documents is enough to trigger the signal but all were generated to prove that all the disjunction
branches are correctly visited.

In [ ]:
emit('''
    any where true
''')

[{}]

In [ ]:
emit('''
    any where not false
''')

[{}]

In [ ]:
emit('''
    any where not (true and false)
''')

[{}]

In [ ]:
emit('''
    any where not (false or false)
''')

[{}]

In [ ]:
emit('''
    network where source.port > 512 and source.port < 1024
''')

[{'event': {'category': ['network']}, 'source': {'port': 859}}]

In [ ]:
emit('''
    network where not (source.port > 512 and source.port < 1024)
''')

[{'event': {'category': ['network']}, 'source': {'port': 236}},
{'event': {'category': ['network']}, 'source': {'port': 19581}}]

In [ ]:
emit('''
    network where source.port > 512 or source.port < 1024
''')

[{'event': {'category': ['network']}, 'source': {'port': 44068}},
{'event': {'category': ['network']}, 'source': {'port': 609}}]

In [ ]:
emit('''
    network where not (source.port < 512 or source.port > 1024)
''')

[{'event': {'category': ['network']}, 'source': {'port': 815}}]

In [ ]:
emit('''
    network where source.port < 2000 and (source.port > 512 or source.port > 1024)
''')

[{'event': {'category': ['network']}, 'source': {'port': 630}},
{'event': {'category': ['network']}, 'source': {'port': 1957}}]

In [ ]:
emit('''
    network where (source.port > 512 or source.port > 1024) and source.port < 2000
''')

[{'event': {'category': ['network']}, 'source': {'port': 1105}},
{'event': {'category': ['network']}, 'source': {'port': 1448}}]

In [ ]:
emit('''
    network where (source.port > 1024 or source.port < 2000) and (source.port < 4000 or source.port > 512)
''')

[{'event': {'category': ['network']}, 'source': {'port': 2567}},
{'event': {'category': ['network']}, 'source': {'port': 569}},
{'event': {'category': ['network']}, 'source': {'port': 61845}},
{'event': {'category': ['network']}, 'source': {'port': 1670}}]

In [ ]:
emit('''
    network where destination.port in (80, 443)
''')

[{'event': {'category': ['network']}, 'destination': {'port': 80}},
{'event': {'category': ['network']}, 'destination': {'port': 443}}]

In [ ]:
emit('''
    network where destination.port not in (80, 443)
''')

[{'event': {'category': ['network']}, 'destination': {'port': 35106}}]

In [ ]:
emit('''
    network where not destination.port in (80, 443)
''')

[{'event': {'category': ['network']}, 'destination': {'port': 58630}}]

In [ ]:
emit('''
    network where destination.port == 22 and destination.port in (80, 443) or destination.port == 25
''')

[{'event': {'category': ['network']}, 'destination': {'port': 25}}]

In [ ]:
emit('''
    process where process.name == "regsvr32.exe"
''')

[{'event': {'category': ['process']}, 'process': {'name': 'regsvr32.exe'}}]

In [ ]:
emit('''
    process where process.name != "regsvr32.exe"
''')

[{'event': {'category': ['process']}, 'process': {'name': 'pki'}}]

In [ ]:
emit('''
    process where process.pid != 0
''')

[{'event': {'category': ['process']}, 'process': {'pid': 1565416049}}]

In [ ]:
emit('''
    process where process.pid >= 0
''')

[{'event': {'category': ['process']}, 'process': {'pid': 2413373806}}]

In [ ]:
emit('''
    process where process.pid > 0
''')

[{'event': {'category': ['process']}, 'process': {'pid': 57239544}}]

In [ ]:
emit('''
    process where process.code_signature.exists == true
''')

[{'event': {'category': ['process']}, 'process': {'code_signature': {'exists': True}}}]

In [ ]:
emit('''
    process where process.code_signature.exists != true
''')

[{'event': {'category': ['process']}, 'process': {'code_signature': {'exists': False}}}]

In [ ]:
emit('''
    any where network.protocol == "some protocol"
''')

[{'network': {'protocol': 'some protocol'}}]

In [ ]:
emit('''
    any where process.pid == null
''')

[{}]

In [ ]:
emit('''
    any where not process.pid != null
''')

[{}]

In [ ]:
emit('''
    any where process.pid != null
''')

[{'process': {'pid': 3617084353}}]

In [ ]:
emit('''
    any where not process.pid == null
''')

[{'process': {'pid': 3003800358}}]

In [ ]:
emit('''
    process where process.name == "regsvr32.exe" and process.parent.name == "cmd.exe"
''')

[{'event': {'category': ['process']}, 'process': {'name': 'regsvr32.exe', 'parent': {'name': 'cmd.exe'}}}]

In [ ]:
emit('''
    process where process.name == "regsvr32.exe" or process.parent.name == "cmd.exe"
''')

[{'event': {'category': ['process']}, 'process': {'name': 'regsvr32.exe'}},
{'event': {'category': ['process']}, 'process': {'parent': {'name': 'cmd.exe'}}}]

In [ ]:
emit('''
    process where process.name == "regsvr32.exe" or process.name == "cmd.exe" or process.name == "powershell.exe"
''')

[{'event': {'category': ['process']}, 'process': {'name': 'regsvr32.exe'}},
{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}},
{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}}]

In [ ]:
emit('''
    process where process.name in ("regsvr32.exe", "cmd.exe", "powershell.exe")
''')

[{'event': {'category': ['process']}, 'process': {'name': 'regsvr32.exe'}},
{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}},
{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}}]

In [ ]:
emit('''
    process where process.name in ("regsvr32.exe", "cmd.exe") or process.name == "powershell.exe"
''')

[{'event': {'category': ['process']}, 'process': {'name': 'regsvr32.exe'}},
{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}},
{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}}]

In [ ]:
emit('''
    process where process.name : ("*.EXE", "*.DLL")
''')

[{'event': {'category': ['process']}, 'process': {'name': 'hhkrsftx.dll'}}]

In [ ]:
emit('''
    process where event.type in ("start", "process_started") and process.args : "dump-keychain" and process.args : "-d"
''')

[{'event': {'category': ['process'], 'type': ['start']}, 'process': {'args': ['dump-keychain', '-d']}},
{'event': {'category': ['process'], 'type': ['process_started']}, 'process': {'args': ['dump-keychain', '-d']}}]

In [ ]:
emit('''
    network where destination.ip == "127.0.0.1"
''')

[{'event': {'category': ['network']}, 'destination': {'ip': '127.0.0.1'}}]

In [ ]:
emit('''
    network where cidrMatch(destination.ip, "10.0.0.0/8", "192.168.0.0/16")
''')

[{'event': {'category': ['network']}, 'destination': {'ip': '192.168.129.181'}}]

In [ ]:
emit('''
    network where not cidrMatch(destination.ip, "10.0.0.0/8", "192.168.0.0/16")
''')

[{'event': {'category': ['network']}, 'destination': {'ip': '106.218.221.201'}}]

In [ ]:
emit('''
    network where destination.ip == "::1"
''')

[{'event': {'category': ['network']}, 'destination': {'ip': '::1'}}]

In [ ]:
emit('''
    network where destination.ip == "822e::/16"
''')

[{'event': {'category': ['network']}, 'destination': {'ip': '822e:d242:3361:b181:c4c:ee59:cfdb:60aa'}}]

## Sequence queries

Following queries instead require multiple _minimal matching documents_, it's not only the content of
a single document that is analyzed but also the relation with the subsequent ones. Therefore a senquence
of documents, with the appropriate relations, is generated each time and all the documents in the sequence
are required for the signal to be generated.

In [ ]:
emit('''
    sequence
        [process where process.name : "cmd.exe"]
        [process where process.parent.name : "cmd.exe"]
''')

[{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}},
{'event': {'category': ['process']}, 'process': {'parent': {'name': 'cmd.exe'}}}]

In [ ]:
emit('''
    sequence by user.id
        [process where process.name : "cmd.exe"]
        [process where process.parent.name : "cmd.exe"]
''')

[{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}, 'user': {'id': 'xgG'}},
{'event': {'category': ['process']}, 'process': {'parent': {'name': 'cmd.exe'}}, 'user': {'id': 'xgG'}}]

In [ ]:
emit('''
    sequence
        [process where process.name : "cmd.exe"] by user.id
        [process where process.parent.name : "cmd.exe"] by user.name
''')

[{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}, 'user': {'id': 'Eev'}},
{'event': {'category': ['process']}, 'process': {'parent': {'name': 'cmd.exe'}}, 'user': {'name': 'Eev'}}]

In [ ]:
emit('''
    sequence
        [process where process.name : "cmd.exe"]
        [process where process.parent.name : "cmd.exe" or process.name : "powershell.exe"]
''')

[{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}},
{'event': {'category': ['process']}, 'process': {'parent': {'name': 'cmd.exe'}}},
{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}},
{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}}]

In [ ]:
emit('''
    sequence by user.id
        [process where process.name : "cmd.exe"]
        [process where process.parent.name : "cmd.exe" or process.name : "powershell.exe"]
''')

[{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}, 'user': {'id': 'GuM'}},
{'event': {'category': ['process']}, 'process': {'parent': {'name': 'cmd.exe'}}, 'user': {'id': 'GuM'}},
{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}, 'user': {'id': 'etd'}},
{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}, 'user': {'id': 'etd'}}]

In [ ]:
emit('''
    sequence
        [process where process.name in ("cmd.exe", "powershell.exe")] by process.name
        [process where process.name in ("cmd.exe", "powershell.exe")] by process.parent.name
''')

[{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}},
{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe', 'parent': {'name': 'cmd.exe'}}},
{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}},
{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe', 'parent': {'name': 'cmd.exe'}}},
{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}},
{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe', 'parent': {'name': 'powershell.exe'}}},
{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}},
{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe', 'parent': {'name': 'powershell.exe'}}}]

In [ ]:
emit('''
    sequence by user.id
        [process where process.name in ("cmd.exe", "powershell.exe")] by process.name
        [process where process.name in ("cmd.exe", "powershell.exe")] by process.parent.name
''')

[{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}, 'user': {'id': 'Tkx'}},
{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe', 'parent': {'name': 'cmd.exe'}}, 'user': {'id': 'Tkx'}},
{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe'}, 'user': {'id': 'REt'}},
{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe', 'parent': {'name': 'cmd.exe'}}, 'user': {'id': 'REt'}},
{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}, 'user': {'id': 'dLh'}},
{'event': {'category': ['process']}, 'process': {'name': 'cmd.exe', 'parent': {'name': 'powershell.exe'}}, 'user': {'id': 'dLh'}},
{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe'}, 'user': {'id': 'Bvu'}},
{'event': {'category': ['process']}, 'process': {'name': 'powershell.exe', 'parent': {'name': 'powershell.exe'}}, 'user': {'id': 'Bvu'}}]

## Error conditions

Not all the queries make sense, for those that cannot logically be ever triggered no single or sequence
of documents can possibly be generated. In such cases an error is reported, as the following cells show.

Of course you can challenge the generation engine first hand and see if the due errors are reported and
make all sense to you.

In [ ]:
emit('''
    any where false
''')

Cannot trigger with any document

In [ ]:
emit('''
    any where not true
''')

Cannot trigger with any document

In [ ]:
emit('''
    any where not (true and true)
''')

Cannot trigger with any document

In [ ]:
emit('''
    any where not (true or false)
''')

Cannot trigger with any document

In [ ]:
emit('''
    any where process.pid == null and process.pid != null
''')

Unsolvable constraints: process.pid (cannot be non-null)

In [ ]:
emit('''
    any where process.pid > 0 and process.pid == null
''')

Unsolvable constraints: process.pid (cannot be null)

In [ ]:
emit('''
    any where process.name != null and process.name == null
''')

Unsolvable constraints: process.name (cannot be null)

In [ ]:
emit('''
    any where process.name == "cmd.exe" and process.name == null
''')

Unsolvable constraints: process.name (cannot be null)

In [ ]:
emit('''
    process where process.pid == 0
''')

Unsolvable constraints: process.pid (out of boundary, 1 <= 0 <= 4294967295)

In [ ]:
emit('''
    process where process.pid <= 0
''')

Unsolvable constraints: process.pid (empty solution space, 1 <= x <= 0)

In [ ]:
emit('''
    process where process.pid < 0
''')

Unsolvable constraints: process.pid (empty solution space, 1 <= x <= -1)

In [ ]:
emit('''
    any where network.protocol == "http" and network.protocol == "https"
''')

Unsolvable constraints ==: network.protocol (is already 'http', cannot set to 'https')

In [ ]:
emit('''
    network where destination.port == 22 and destination.port in (80, 443)
''')

Cannot trigger with any document

In [ ]:
emit('''
    network where not (source.port > 512 or source.port < 1024)
''')

Unsolvable constraints: source.port (empty solution space, 1024 <= x <= 512)

In [ ]:
emit('''
    sequence by process.name
        [process where process.name : "cmd.exe"]
        [process where process.name : "powershell.exe"]
''')

Unsolvable constraints ==: process.name (is already 'powershell.exe', cannot set to 'cmd.exe')

In [ ]:
emit('''
    sequence
        [process where process.name : "cmd.exe"] by process.name
        [process where process.parent.name : "powershell.exe"] by process.parent.name
''')

Unsolvable constraints ==: process.parent.name (is already 'powershell.exe', cannot set to 'cmd.exe')

In [ ]:
emit('''
    sequence by process.name
        [process where process.name == null]
        [process where process.name : "powershell.exe"]
''')

Unsolvable constraints: process.name (cannot be non-null)

## Any oddities?

Did you find anything odd reviewing the report or playing with the documents emitter?
We are interested to know.